<a href="https://colab.research.google.com/github/tizwe/Image-restoration-via-a-DnGAN-and-DMSP/blob/main/Image_Restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Please ensure GPU and High RAM  enabled in the Runtime Settting
"""

!pip install tfa-nightly    #To allow us to import SpectralNormalization

import numpy as np
import time 
import itertools
import random
import matplotlib.pyplot as plt
import scipy.io as sio
import tensorflow as tf 
import tensorflow_datasets as  tfds
from tensorflow.data import Dataset as Set
from tensorflow_addons.layers import SpectralNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, Reshape
from tensorflow.keras.layers import Lambda, LeakyReLU, PReLU, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.image import psnr
from scipy import ndimage, signal as sig
from skimage.metrics import structural_similarity as ssim
from IPython.display import clear_output
from PIL import Image


In [2]:
#Hyperparaameters

sigma=0.1
BUFFER_SIZE = 60000
batch_size = 8
num_epochs = 50 
num_examples_to_generate = 6

In [ ]:
"""
Currently (19 Dec 2020) the website where our dataset was pulled from
is down and hence the import fails. 
This will hopefully be resolved in a few days.
"""

#Import the lfw dataset which contains images of faces
dataset = tfds.builder('lfw')
dataset.download_and_prepare()
dataset = dataset.as_dataset()
dataset = dataset['train']

# Converting Dataset to a NumPy Array

lfw=[]

for example in tfds.as_numpy(dataset):
  image= example['image']
  image=image[35:-35,35:-35,:]
  lfw.append(image)

lfw=np.array(lfw).astype(np.float32)


# Split the data into traing set and test set which will be used for evaluation
# We have to throw some parts of the dataet aways,
# due to limited RAM capacity

take_p=0.4 
random.shuffle(lfw)
(x_train,x_test,_)=np.split(lfw,[int(lfw.shape[0]*take_p),
                                 int(lfw.shape[0]*(take_p+0.05))])


# Bring the image date into the desired format for training

im_dim1 = x_train[1].shape[0]     
im_dim2 = x_train[1].shape[1]

x_train = np.reshape(x_train, [-1, im_dim1, im_dim2, 3])
x_test = np.reshape(x_test, [-1, im_dim1, im_dim2, 3])
x_train = x_train / 255
x_test = x_test / 255


# Create noisy images
# Train Images
noise=np.random.normal(loc=0,scale=sigma, size=x_train.shape).astype(np.float32)
x_train_noisy = x_train + noise
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
del noise  
#TestImages
noise=np.random.normal(loc=0, scale=sigma, size=x_test.shape).astype(np.float32)
x_test_noisy = x_test + noise
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
del noise #Delete Noise to free up Space in RAM 


# Split Dataset into Batches
train_dataset_clean = Set.from_tensor_slices(x_train).batch(batch_size)
train_dataset_noisy = Set.from_tensor_slices(x_train_noisy).batch(batch_size)


In [ ]:


# kernel size for the convolution
kernel_size=3  
l_pad,r_pad=int(np.floor((kernel_size-1)/2)),int(np.ceil((kernel_size-1)/2))

# Scales the padding size withrespect to the kernel size
pad_shape=([0,0],[l_pad,r_pad],[l_pad,r_pad],[0,0])


"""
Manually pad before every Convolution to maintain the shape.
This is because altought the Conv2D function can also perform 
shape-maintaing convolution, it does so by 0-padding. (Adding 0 at the borders)
However, this created artifacts at the border of the images.
With symmetric padding (duplicate border pixels)
those artifacts can be reduced.
"""


def make_generator_model(input_shape=(None,None,3)):

    inpt= Input(shape=(input_shape[0],input_shape[1],input_shape[2]))
    h0=   tf.pad(inpt,([0,0],[4,4],[4,4],[0,0]),'symmetric')
    h0=   Conv2D(64,(9, 9),padding='valid')(h0)
    out1= PReLU()(h0)


    h1= tf.pad(out1,pad_shape,'symmetric')
    h1= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h1)
    h1= BatchNormalization()(h1)
    h1= PReLU()(h1)
    h1= tf.pad(h1,pad_shape,'symmetric')
    h1= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h1)
    h1= BatchNormalization()(h1)
    out2=out1+h1

    h2= tf.pad(out2,pad_shape,'symmetric')
    h2= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h2)
    h2= BatchNormalization()(h2)
    h2= PReLU()(h2)
    h2= tf.pad(h2,pad_shape,'symmetric')
    h2= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h2)
    h2= BatchNormalization()(h2)
    out3= out2+h2

    h3= tf.pad(out3,pad_shape,'symmetric')
    h3= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h3)
    h3= BatchNormalization()(h3)
    h3= PReLU()(h3)
    h3= tf.pad(h3,pad_shape,'symmetric')
    h3= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h3)
    h3= BatchNormalization()(h3)
    out4=out3+h3

    h4= tf.pad(out4,pad_shape,'symmetric')
    h4= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h4)
    h4= BatchNormalization()(h4)
    h4= PReLU()(h4)
    h4= tf.pad(h4,pad_shape,'symmetric')
    h4= SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h4)
    h4= BatchNormalization()(h4)
    out5=out4+h4


    h5=tf.pad(out5,pad_shape,'symmetric')
    h5=SpectralNormalization(
        Conv2D(64,(kernel_size, kernel_size), padding='valid'))(h5)
    h5=BatchNormalization()(h5)
    out6=out1+h5

    h6=tf.pad(out6,pad_shape,'symmetric')
    h6=Conv2D(256,(kernel_size, kernel_size), padding='valid')(h6)
    out7=PReLU()(h6)

    h7=tf.pad(out7,pad_shape,'symmetric')
    h7=Conv2D(256,(kernel_size, kernel_size), padding='valid')(h7)
    out8=PReLU()(h7)

    h8=tf.pad(out8,([0,0],[4,4],[4,4],[0,0]),'symmetric')
    output=Conv2D(3,(9, 9), padding='valid',activation='sigmoid')(h8)
    model=Model(inputs=[inpt], outputs=output)

    return model

generator = make_generator_model(input_shape=(im_dim1,im_dim2,3))
dae = make_generator_model(input_shape=(im_dim1,im_dim2,3))


def make_discriminator_model(input_shape=(None,None,3)):
  
    inpt=Input(shape=(input_shape[0],input_shape[1],input_shape[2]))

    h1= Reshape((im_dim1,im_dim2,3))(inpt)
    h1= SpectralNormalization(Conv2D(64, (3, 3), strides=(1, 1)))(inpt)
    out1= LeakyReLU()(h1)


    h2= SpectralNormalization(Conv2D(64, (3, 3), strides=(2, 2)))(out1)
    h2= LayerNormalization()(h2)
    out2=LeakyReLU()(h2)

    h3= SpectralNormalization(Conv2D(128, (3, 3), strides=(1, 1)))(out2)
    h3= LayerNormalization()(h3)
    out3=LeakyReLU()(h3)

    h4= SpectralNormalization(Conv2D(128, (3, 3), strides=(2, 2)))(out3)
    h4= LayerNormalization()(h4)
    out4=LeakyReLU()(h4)

    h5= SpectralNormalization(Conv2D(256, (3, 3), strides=(1, 1)))(out4)
    h5= LayerNormalization()(h5)
    out5=LeakyReLU()(h5)

    h6= SpectralNormalization(Conv2D(256, (3, 3), strides=(2, 2)))(out5)
    h6= LayerNormalization()(h6)
    out6=LeakyReLU()(h6)
    
    h7= SpectralNormalization(Conv2D(256, (3, 3), strides=(1, 1)))(out6)
    h7= LayerNormalization()(h7)
    out7=LeakyReLU()(h7)

    h8= SpectralNormalization(Conv2D(256, (3, 3), strides=(2, 2)))(out7)
    h8= LayerNormalization()(h8)
    out8=LeakyReLU()(h8)
    

    h9=Flatten()(out8)
    h9=Dense(1024)(h9)
    out9=LeakyReLU()(h9)
  
    output= Dense(1,activation='sigmoid')(out9)

    model=Model(inputs=[inpt], outputs=output)

    return model



discriminator = make_discriminator_model(input_shape=(im_dim1,im_dim2,3))




In [ ]:
"""
To import already trained models, uncomment the following lines:
"""

#generator=tf.keras.models.load_model('generator.h5') 
#discriminator=tf.keras.models.load_model('discriminator.h5')
#dae=tf.keras.models.load_model('keras.h5')

In [ ]:
#Seed for visualization
inde=np.random.randint(x_test.shape[0],size=num_examples_to_generate)
seed=x_test_noisy[0:num_examples_to_generate]
seed_clear=x_test[0:num_examples_to_generate]

In [ ]:
#Define the loss functions as the cross-entropy loss

def discriminator_loss(real_output, fake_output):

    real_loss=tf.math.log(real_output)
    fake_loss=tf.math.log(1.-fake_output)

    return -tf.reduce_mean(real_loss+fake_loss)

def generator_loss(fake_output):

    fake_loss=-tf.reduce_mean(tf.math.log(fake_output))

    return fake_loss

#Eucledian loss fot the DAE
def dae_loss(gen_img,real_img):

    e_loss=tf.reduce_mean(tf.nn.l2_loss(gen_img-real_img))

    return  e_loss


#Increasing learning rate naturally increases the training speed
#But models are more likely to collaps then  

generator_optimizer = tf.keras.optimizers.Adam(0.0000005)
disc_optimizer = tf.keras.optimizers.Adam(0.0000005)
dae_optimizer = tf.keras.optimizers.Adam(0.000001)




In [ ]:
#auxiliary functions

def img_standardization(img):
  return tf.image.per_image_standardization(img)

def nrm(x):
  return (x-np.min(x))/(np.ptp(x))

def stan(x):
  x=np.expand_dims(x,0)
  x=tf.map_fn(lambda img: tf.image.per_image_standardization(img), x)
  return x[0].numpy()

In [ ]:
def train_step_dngan(clean_images,nsy_images):
  """
  Implements a gradient descent step for the DnGAN 
  """

  # Take the first half of the clean images 
  # and the second half of the noisy images 
  # to ensure the dicriminator never sees 
  # both versions of the same image

  half_size=(np.ceil(clean_images.shape[0] / 2)).astype(int)

  real_images=clean_images[ :half_size]
  noisy_images=nsy_images[-half_size: ]


  # Gradient Step

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

    generated_images = generator(noisy_images, training=True)
    generated_images = tf.map_fn(img_standardization, generated_images)
    real_output = discriminator(real_images, training=True)
    fake_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(fake_output) 
    disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss,
                                                generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator,
                                            generator.trainable_variables))
    gradients_of_disc = disc_tape.gradient(disc_loss,
                                          discriminator.trainable_variables)
    disc_optimizer.apply_gradients(zip(gradients_of_disc,
                                       discriminator.trainable_variables))
                                                

       

In [ ]:
def train_step_dae(clean_images,nsy_images):
  """
  Implements the gradient descent steps for the DAE 
  """
  real_images=clean_images
  noisy_images=nsy_images

  with tf.GradientTape() as dae_tape:
    
    generated_images = dae(noisy_images, training=True)
    generated_images = tf.map_fn(img_standardization, generated_images)
    dae_l = dae_loss(generated_images, real_images) 
    gradients_of_dae = dae_tape.gradient(dae_l, dae.trainable_variables)
    dae_optimizer.apply_gradients(zip(gradients_of_dae, 
                                      dae.trainable_variables))


In [ ]:
def train(clean_dataset, noisy_dataset, epochs=100):
  """
  Start the training process
  """
  for epoch in range(epochs):

    start = time.time()
    for img_batch_clean,img_batch_nsy in zip(clean_dataset,noisy_dataset):
      
      img_batch_clean=tf.map_fn(img_standardization,
                              img_batch_clean)
      img_batch_nsy=tf.map_fn(img_standardization,
                             img_batch_nsy)
      
      img_batch_clean=tf.convert_to_tensor(img_batch_clean)
      img_batch_nsy=tf.convert_to_tensor(img_batch_nsy)
      train_step_dngan(img_batch_clean,img_batch_nsy)

      #Since the DAE trains much faster, we only train it every second epoch
      if epoch % 2 ==0:
        train_step_dae(img_batch_clean,img_batch_nsy)


    clear_output(wait=True)

    generate_images([generator,dae],
                    epoch,
                    seed)
    
    tm= time.time()-start

    print (f'Time for epoch {epoch} is {tm} sec')

    # Save the model every 5 epochs
    if epoch%5==0:
      generator.save('gen'+str(epoch)+'.h5')
      discriminator.save('disc'+str(epoch)+'.h5')
      dae.save('dae'+str(epoch)+'.h5')

      
  generator.save('generator.h5')
  discriminator.save('discriminator.h5')
  dae.save(('dae.h5'))


  # Generate after the final epoch
  clear_output(wait=True)
  generate_images([generator,dae],
                  epochs,
                  seed)

In [ ]:
def generate_images(models, epoch, test_input):
  # Notice `training` is set to False.

  #number of images used to evaluate the performance
  #on the test set

  sbtch=30  # Number is limited by RAM capacity

  for model, ssim_hist, psnr_hist in zip(models,ssim_list,psnr_list): 

    predictions = model(tf.map_fn(img_standardization, 
                                  test_input), training=False)
    predictions=tf.map_fn(nrm,predictions)

    predictions2 = model(tf.map_fn(img_standardization, x_test_noisy[0:sbtch]),
                        training=False)
    predictions2=tf.map_fn(nrm,predictions2)
  
    ssim_val=np.mean([
                  ssim(np.array(predictions2[i]),
                        x_test[i],multichannel=True) 
                  for i in range(sbtch) 
                  ])
    psnr_val=np.mean([
                  psnr(np.array(predictions2[i]),
                      x_test[i],1)
                  for i in range(sbtch)
                  ])
        
    ssim_hist.append(ssim_val)
    psnr_hist.append(psnr_val)

    #Plot (Epoh,SSIM) and (Epoch,PSNR) in the same graph 
    fig,ax1 = plt.subplots()
    ax1.plot(ssim_hist,label='\n ssim')
    plt.legend(loc='upper left')
    ax1.tick_params(axis='y')
    ax2 = ax1.twinx()  
    ax2.plot(psnr_hist,color='r',label='psnr')
    ax2.tick_params(axis='y')
    plt.legend(loc='upper left')
    plt.savefig('psnrh_hist.png')
    plt.show()

    fig = plt.figure(figsize=(80,80))
    for i in range(4):
        plt.subplot(4, 3, i*3+1)
        plt.imshow((predictions[i][:, :, :]))
        plt.axis('off')
        plt.subplot(4, 3, i*3+2)
        plt.imshow((seed[i, :, :,:]))
        plt.axis('off')
        plt.subplot(4, 3, i*3+3)
        plt.imshow(np.clip(seed_clear[i],0,1))
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [4]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c9328b96-4cf0-0d22-d40b-9a021ea2fd42)


In [ ]:
#Lists where we will save the progress during training

ssim_dae_hist=[]
psnr_dae_hist=[]
ssim_dngan_hist=[]
psnr_dngan_hist=[]

ssim_list=[ssim_dae_hist,ssim_dngan_hist]
psnr_list=[psnr_dae_hist,psnr_dngan_hist]

In [ ]:
"""
This may take a while.
To speed things up, consider decreasing the number of training images instead of 
lowerig the number of epochs.
If OOM errors occur, lower the Batch size.
To continue training, run this cell again.
"""
#Note: Throws warnings since Tensorflow 2.4 
#These can be ignored and might likely go away with further updates. 

train(train_dataset_clean,
      train_dataset_noisy,
      200)



# Deblur


The following work, is a derivative of [Deep Mean-Shift Priors for Image Restoration](https://github.com/siavashBigdeli/DMSP-tensorflow#deep-mean-shift-priors-for-image-restoration-project-page) by Bigdelli et al., used under CC BY-NC-SA 4.0. This work is hence also licensed under CC BY-NC-SA 4.0 by Tizian Weber.

In [ ]:
# Adjust the denoiser to fit in the needed Input/Output format


def den_dae(x):
  x=np.expand_dims(x,0)
  x=tf.map_fn(lambda img: tf.image.per_image_standardization(img), x)
  x=dae(x)
  x=x[0]
  return x

def den_dngan(x):
  x=np.expand_dims(x,0)
  x=tf.map_fn(lambda img: tf.image.per_image_standardization(img), x)
  x=generator(x)
  x=x[0]
  return x


In [ ]:
def computePSNR(img1, img2, pad_y, pad_x):
    """ Computes peak signal-to-noise ratio between two images. 
    Input:
    img1: First image in range of [0, 255].
    img2: Second image in range of [0, 255].
    pad_y: Scalar radius to exclude boundaries from contributing to PSNR computation in vertical direction.
    pad_x: Scalar radius to exclude boundaries from contributing to PSNR computation in horizontal direction.
    
    Output: PSNR """

    # Crop the edges away that were added by padding
    # '...or  None' to catch the '-0' Case.
    img1=img1[pad_y:-pad_y or None,
              pad_x:-pad_x or None,
              :]

    img2=img2[pad_y:-pad_y or None,
              pad_x:-pad_x or None,
              :]
    img1=nrm(stan(img1))
    img2=nrm(stan(img2))
    psnr=tf.image.psnr(img1,img2,1)
    ssm=ssim(img1,img2,multichannel=True)
    return psnr,ssm

def filter_image(image, kernel, mode='valid'):
    """ Implements color filtering (convolution using a flipped kernel) """
    chs = []
    for d in range(image.shape[2]):
        channel = sig.convolve2d(image[:,:,d], 
                                 np.flipud(np.fliplr(kernel)), mode=mode)
        chs.append(channel)
    return np.stack(chs, axis=2)

def convolve_image(image, kernel, mode='valid'):
    """ Implements color image convolution """
    chs = []
    for d in range(image.shape[2]):
        channel = sig.convolve2d(image[:,:,d], kernel, mode=mode)
        chs.append(channel)
    return np.stack(chs, axis=2)

def DMSPDeblur(degraded, kernel, sigma_d, params):
    """ Implements stochastic gradient descent (SGD) 
    Bayes risk minimization for image deblurring described in:
     "Deep Mean-Shift Priors for Image Restoration" 
     (http://home.inf.unibe.ch/~bigdeli/DMSPrior.html)
     S. A. Bigdeli, M. Jin, P. Favaro, M. Zwicker, 
     Advances in Neural Information Processing Systems (NIPS), 2017 
     
     Input:
     degraded: Observed degraded RGB input image in range of [0, 255].
     kernel: Blur kernel (internally flipped for convolution).
     sigma_d: Noise standard deviation. (set to -1 for noise-blind deblurring)
     params: Set of parameters.
     params.denoiser: The denoiser function hanlde.
    
     Optional parameters:
     params.sigma_dae:  The Standard deviation of the denoiser 
                        training noise. default: 11
     params.num_iter: Specifies number of iterations.
     params.mu: The momentum for SGD optimization. default: 0.9
     params.alpha the step length in SGD optimization. default: 0.1
    
     Outputs:
     res: Solution."""


    if 'denoiser' not in params:
        raise ValueError('Need a denoiser in params.denoiser!')
        
    if 'gt' in params and params['print'] is True:
        print_iter = True
    else:
        print_iter = False
    
    if 'sigma_dae' not in params:
        params['sigma_dae'] = 11.0

    if 'num_iter' not in params:
        params['num_iter'] = 10
        
    if 'mu' not in params:
        params['mu'] = 0.9
    
    if 'alpha' not in params:
        params['alpha'] = 0.1

    sigma_dae=params['sigma_dae']


    pad_y = np.floor(kernel.shape[0] / 2.0).astype(np.int64)
    pad_x = np.floor(kernel.shape[1] / 2.0).astype(np.int64)
    res = np.pad(degraded, pad_width=((pad_y, pad_y), (pad_x, pad_x), (0, 0)), 
                 mode='edge')
    step = np.zeros(res.shape)

    # Calculate some expressions now 
    # to avoid redundant calculations in the loop

    sigma_sq = sigma_dae**2
    if  sigma_d >= 0:
        relative_weight = (1/ sigma_d**2 ) / ( 1/ sigma_d**2 + 1/ sigma_sq)
    term=degraded.size * 2*sigma_sq * (np.sum(np.power(kernel[:], 2)))
    

    histp=[]    # PSNR Hist
    hists=[]    # SSIM Hist

    if print_iter:
        psnr,ssm = computePSNR(params['gt'], res, pad_y, pad_x)
        print(f'Initialized with PSNR: {psnr:.4f} SSIM : {ssm:.4f} ' )

    for iter in range(params['num_iter']):
        if print_iter:
            t = time.time()

        #     compute prior gradient
        noise = np.random.normal(0.0,sigma_dae, res.shape).astype(np.float32)
        rec = params['denoiser'](res + noise)
        rec=rec.numpy()
        rec=nrm(rec)
        prior_grad = res - rec

        #     compute data gradient
        map_conv = filter_image(res, kernel)
        data_err = map_conv - degraded
        data_grad = convolve_image(data_err, kernel, mode='full')

        if sigma_d < 0:
            lambda_ = (degraded.size) / (
                        np.sum(np.power(data_err[:], 2)) + term)
            relative_weight = lambda_ / (lambda_ + 1 / sigma_sq)

        #     sum the gradients

        grad_joint = data_grad * relative_weight + prior_grad * (1 - relative_weight)

        #     update
        step = params['mu'] * step - params['alpha'] * grad_joint
        res = res + step
        res = np.minimum(1.0, np.maximum(0, res)).astype(np.float32)

        if print_iter:
            
            psnr,ssm  = computePSNR(params['gt'], res, pad_y, pad_x)
            hists.append(ssm)
            histp.append(psnr)
            it_time=time.time()-t
            if iter % 5 == 0:
              print('Finished iteration: ' + str(iter))
              print (f'PSNR: {psnr:.4f}, SSIM: {ssm:.4f}, iteration finished in {it_time} seconds')
                   



    #Plot the PSNR and SSIM Values over epochs 
    fig,ax1 = plt.subplots()
    ax1.plot(histp,label='psnr')
    ax1.tick_params(axis='y')
    ax2 = ax1.twinx()  
    ax2.plot(hists,color='r',label='ssim')
    ax2.tick_params(axis='y')
    fig.legend()
    plt.show()
    return res

In [ ]:
def deblur_setup(gt,sigma_d,denoiser,num_iter=100,print_iter=True):
  """
  Input:
  gt: The ground truth image 
  sigma_d: The SD of the noise to be added 
  denoiser: The denoiser used in the DMSP 
  num_iter: Number of iterations
  print_iter: whether intermed. results should be printed. default: True
  Output: 

  psnrvalues: PSNR values of the results
  ssimvalues: SSIM values of the results 
  im: Images normalized to [0,1]      
  """


  # The DMSP deblur function and the RGB filtering function (flipped convolution)
  # The denoiser implementation


  denoise = denoiser


  # Load data
  sigma_d = sigma_d/255

  # Load kernel (kernels.mat must be in the same directory as this notebook)
  # If running on Colab, just drag-drop kernels.mat  in the folder symbol on the right 
  ms = sio.loadmat('kernels.mat')
  k=ms['kernels']
  kernel=k[0][1]  # Choose a kernel
  k_dec=7  # Decrease the kernel size
  kernel=kernel[k_dec:-k_dec, k_dec:-k_dec]
  kernel= kernel/np.sum(kernel)
  degraded = filter_image(gt, kernel)
  noise = np.random.normal(0.0, sigma_d, degraded.shape).astype(np.float32)
  degraded = degraded + noise
  degraded=np.clip(degraded,0,1)



  # non-blind deblurring
  # run DMSP

  params = {}
  params['denoiser'] = denoise
  params['sigma_dae'] = 0.1
  params['num_iter'] = num_iter
  params['mu'] = 0.8
  params['alpha'] = 0.03
  params['gt'] = gt # feed ground truth to monitor the PSNR at each iteration
  params['print'] = print_iter
  restored = DMSPDeblur(degraded, kernel, sigma_d, params)

  img_restored = Image.fromarray((np.clip(restored, 0, 1)*255).astype(dtype=np.uint8))


  # noise-blind deblurring
  # run DMSP noise-blind

  params = {}
  params['denoiser'] = denoise
  params['sigma_dae'] = 0.1
  params['num_iter'] = num_iter
  params['mu'] = 0.8
  params['alpha'] = 0.03
  params['gt'] = gt
  params['print'] = print_iter

  restored_nb = DMSPDeblur(degraded, kernel, -1, params)


  # Generate the images and calculate PSNR/SSIM
  
  img_degraded = Image.fromarray((np.clip(degraded, 0, 1)*255).astype(dtype=np.uint8))
  img_restored_nb = Image.fromarray((np.clip(restored_nb, 0, 1)*255).astype(dtype=np.uint8))

  psnrvalues=[]
  ssimvalues=[]

  pad_y = np.floor(kernel.shape[0] / 2.0).astype(np.int64)
  pad_x = np.floor(kernel.shape[1] / 2.0).astype(np.int64)
  psnr, ssim = computePSNR(gt, 
                   np.pad(degraded, pad_width=((pad_y, pad_y), (pad_x, pad_x), (0, 0)), mode='edge').astype(np.float32), 
                   pad_y, pad_x)  
  
  psnrvalues.append(psnr)
  ssimvalues.append(ssim)

  psnr, ssim = computePSNR(gt, restored, pad_y, pad_x)
  psnrvalues.append(psnr)
  ssimvalues.append(ssim)

  psnr, ssim = computePSNR(gt, restored_nb, pad_y, pad_x)
  psnrvalues.append(psnr)
  ssimvalues.append(ssim)

  psnr, ssim = computePSNR(gt,denoise(restored), pad_y, pad_x)
  psnrvalues.append(psnr)
  ssimvalues.append(ssim)

  psnr, ssim = computePSNR(gt, denoise(restored_nb), pad_y, pad_x)
  psnrvalues.append(psnr)
  ssimvalues.append(ssim)

  im=[] 
  im.append(gt)
  im.append(degraded)
  im.append(restored)
  im.append(restored_nb)
  im.append(nrm(denoise(restored)))
  im.append(nrm(denoise(restored_nb)))

  return psnrvalues,ssimvalues, im 

In [ ]:
im=random.choice(x_train) #Pick a random image to test 
sigm=5 

print("Start deblurring with DAE")
dae_p,dae_s,im_dae=deblur_setup(im,sigm,den_dae,200)
print("Start deblurring with DnGAN")
dngan_p,dngan_s,im_dngan=deblur_setup(im,sigm,den_dngan,200)

In [ ]:
"""
Print and show results 
"""

print(f'Degraded: \t PSNR: {dae_p[0]} , SSIM: {dae_s[0]}')
print(f'DAE: \t PSNR: {dae_p[1]} , SSIM: {dae_s[1]}')
print(f'DAE na: \t PSNR: {dae_p[2]} , SSIM: {dae_s[2]}')
print(f'DnGAN: \t PSNR: {dngan_p[1]} , SSIM: {dngan_s[1]}')
print(f'DnGAN na: \t PSNR: {dngan_p[2]} , SSIM: {dngan_s[2]}')
print(f'DAE +: \t PSNR: {dae_p[3]} , SSIM: {dae_s[3]}')
print(f'DAE na+: \t PSNR: {dae_p[4]} , SSIM: {dae_s[4]}')



fig, axes = plt.subplots(2, 4, constrained_layout=False, figsize=(40, 20))
plt.subplots_adjust(wspace=0, hspace=0)


ax=axes[0,0]
ax.axis('Off')
ax.set_title('Original',fontsize=50)
ax.imshow(im_dngan[0])

ax=axes[0,1]
ax.axis('Off')
ax.set_title('Degraded',fontsize=50)
ax.imshow(im_dngan[1])


ax=axes[0,2]
ax.axis('Off')
ax.set_title('DAE',fontsize=50)
ax.imshow(im_dae[2])

ax=axes[0,3]
ax.axis('Off')
ax.set_title('DAE na',fontsize=50)
ax.imshow(im_dae[3])

ax=axes[1,0]
ax.axis('Off')
ax.set_title('DGAN',fontsize=50,y=-0.1)
ax.imshow(im_dngan[2])

ax=axes[1,1]
ax.axis('Off')
ax.set_title('DGAN na',fontsize=50,y=-0.1)
ax.imshow(im_dngan[3])

ax=axes[1,2]
ax.axis('Off')
ax.set_title('DAE +',fontsize=50,y=-0.1)
ax.imshow(im_dae[4])

ax=axes[1,3]
ax.axis('Off')
ax.set_title('DAE na +',fontsize=50,y=-0.1)
ax.imshow(im_dae[5])
plt.savefig('results_deblur_sig' + str(sigm) + '.png')

plt.show()